### <span style="color:#800000">Web Scraping Question & Answers from Indeed</span>

### <span style="color:#FF00FF">Import libraries</span>

In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

from tqdm.auto import tqdm

pd.set_option('display.max_columns', 500)

### <span style="color:#FF00FF">Extract Roles</span>

In [2]:
BASE_URL = "https://www.indeed.com"
SUB_URL = "https://www.indeed.com/hire/interview-questions"

roles = []

page = requests.get(SUB_URL)
soup = BeautifulSoup(page.text, "html.parser")
postlist = soup.find(name="div", attrs={"id":"jobDesListId"})

for i in postlist.find_all('a', href=True):
    roles.append([i.text,BASE_URL+i['href']])
    

### <span style="color:#FF00FF">Extract Question & Answers</span>

In [3]:
def get_questionanswers(url,domain):
    qa = []
    try: 
        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html.parser")

        questions = soup.find_all(name="div", attrs={"class":"qa-question"})
        
        for q in questions:
            question = q.find(["h1", "h2", "h3"])
            answer = q.find(name="div", attrs={"class":"qa-answer"})

            question =  question.text
            answer  = answer.text
            qa.append([domain,url,question,answer])
    except:
        print(url)

    return qa

In [5]:
qa = []
for i in tqdm(roles):
    domain = i[0]
    url = i[1]
    getqa = get_questionanswers(url,domain)
    qa.extend(getqa)

### <span style="color:#FF00FF">Convert Result to DataFrame</span>

In [6]:
cols = ['roles','url','questions','answers']
qdf = pd.DataFrame(qa,columns=cols)

qdf.sample(n=5)

,roles,url,questions,answers
129,Art Director,https://www.indeed.com/hire/interview-question...,"1. As someone with artistic talent, who is you...",\nNot only is this question a fantastic icebre...
1983,Physical Therapy Assistant,https://www.indeed.com/hire/interview-question...,Do you have experience working with elderly pa...,\nWith society's increasing life expectancy an...
2661,Systems Engineer,https://www.indeed.com/hire/interview-question...,What does DNS mean? How does it work on an int...,\nDNS stands for Domain Name System or Domain ...
723,Cook,https://www.indeed.com/hire/interview-question...,Why have you chosen a career in the field of c...,\nThis question serves at least two purposes. ...
2850,Welder,https://www.indeed.com/hire/interview-question...,Describe prototype tooling and what kind of ro...,\nPrototype tooling is a huge part of working ...


### <span style="color:#FF00FF">Export Dataset</span>

In [9]:
qdf.to_csv("question_answers.csv",index=False)